<a href="https://colab.research.google.com/github/KillerStrike17/10DaysofMLChallenge/blob/master/Day3/Detox_With_TPU_Day_3_10DaysofMLChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day3

## Problem Statement

Analyze the Toxicity of comment's 

Data

! wget https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv

! wget https://www.dropbox.com/s/tst2y6mzwzbhxo3/test.csv

### Importing Library: Calling the Gods

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import numpy as np
import gc
import pandas as pd

TensorFlow 2.x selected.


In [2]:
print(tf.__version__)

2.1.0


In [3]:
#Setting the Tensorflow Environment
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Initializing the TPU system: 10.2.128.250:8470


INFO:tensorflow:Initializing the TPU system: 10.2.128.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


### Getting Dataset

In [4]:
!wget "https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv"

--2020-03-25 14:26:00--  https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ggl9krhh6dcwhhz/train.csv [following]
--2020-03-25 14:26:00--  https://www.dropbox.com/s/raw/ggl9krhh6dcwhhz/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6056360083e0b31fd492ebf37f.dl.dropboxusercontent.com/cd/0/inline/A0mVYSgUmysCPsw3hquiqapxCOzBLzoFheTvMIuY9zh6KAEZ0RPOuVUxTSLxaB-V_kY6fpxdeGbqxx_Ppuapo70LSqhj2FcGjqlIkN9OZH_pqw/file# [following]
--2020-03-25 14:26:00--  https://uc6056360083e0b31fd492ebf37f.dl.dropboxusercontent.com/cd/0/inline/A0mVYSgUmysCPsw3hquiqapxCOzBLzoFheTvMIuY9zh6KAEZ0RPOuVUxTSLxaB-V_kY6fpxdeGbqxx_Ppuapo70LSqhj2FcGjqlIkN9OZH_pqw/file
Resolving uc6056360083e0b31f

In [5]:
!wget "https://www.dropbox.com/s/tst2y6mzwzbhxo3/test.csv"

--2020-03-25 14:26:04--  https://www.dropbox.com/s/tst2y6mzwzbhxo3/test.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tst2y6mzwzbhxo3/test.csv [following]
--2020-03-25 14:26:04--  https://www.dropbox.com/s/raw/tst2y6mzwzbhxo3/test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd1b9c65f3d3c7a3cc662a05cc1.dl.dropboxusercontent.com/cd/0/inline/A0kyKh-Q6p_Upgg8BwwoChKhff_OXwwfyp3_o0o3RR_14NizOLtC1L7tDICEQeS9gA60EnMQXIiR6HiIm1OnYiXE6NqNRDp7sbhqU2V7nYVaBA/file# [following]
--2020-03-25 14:26:04--  https://ucd1b9c65f3d3c7a3cc662a05cc1.dl.dropboxusercontent.com/cd/0/inline/A0kyKh-Q6p_Upgg8BwwoChKhff_OXwwfyp3_o0o3RR_14NizOLtC1L7tDICEQeS9gA60EnMQXIiR6HiIm1OnYiXE6NqNRDp7sbhqU2V7nYVaBA/file
Resolving ucd1b9c65f3d3c7a3cc66

In [6]:
# Code to Mount drive to colab
from google.colab import drive
drive.mount('/content/gdrive')
# Code to copy the data from drive
!cp /content/gdrive/My\ Drive/Dataset/Glove\ Model/glove.6B.zip /content/
!unzip glove.6B.zip

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_x = train['comment_text']
train_y = train[classes].values
test_x = test['comment_text']

In [8]:
train_x.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [9]:
train_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [10]:
test_x.head()

0    Thank you for understanding. I think very high...
1                     :Dear god this site is horrible.
2    "::: Somebody will invariably try to add Relig...
3    " \n\n It says it right there that it IS a typ...
4    " \n\n == Before adding a new product to the l...
Name: comment_text, dtype: object

### Model Building

In [0]:
maxlen = 200 # cuts off reviews after 200 words
max_words = 20000 # Dictionary Limit

In [12]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list(train_x))
train_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences =  tokenizer.texts_to_sequences(test_x)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
X_train = pad_sequences(train_sequences, maxlen=maxlen)
X_test = pad_sequences(test_sequences, maxlen=maxlen)

Found 210337 unique tokens.


In [18]:
glove_dir = '/content'
embedding_dim = 50
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.{}d.txt'.format(embedding_dim)))
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs

f.close()

print('Found %s word vectors.' %len(embeddings_index))
gc.collect()

Found 400000 word vectors.


165

In [0]:
embedding_matrix = np.zeros((max_words, embedding_dim))
# print(max_words)
for word, i in word_index.items():
  if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [0]:
def mymodel():
  input_layer = Input(shape=(maxlen, ))
  x = Embedding(max_words, embedding_dim, input_length=maxlen)(input_layer)
  x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1))(x)
  x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer2',dropout=0.2,recurrent_dropout=0.2))(x)
  x = GlobalMaxPool1D()(x)
  x = Dropout(0.1)(x)
  x = Dense(50, activation="relu")(x)
  x = Dropout(0.1)(x)
  output_layer = Dense(6, activation="sigmoid")(x)
  model = Model(inputs=input_layer, outputs=output_layer)
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model


### Model Training


#### With TPU

In [29]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
  model = mymodel()
model.summary()
history = model.fit(X_train,train_y,epochs=15,batch_size=197 * 8,validation_split=0.20)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 50)           1000000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200, 120)          53280     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200, 120)          86880     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 120)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                6050

In [0]:
model.save_weights('/content/tpu_model.h5', overwrite=True)

In [0]:
inferencing_model = mymodel()
inferencing_model.load_weights('/content/tpu_model.h5')

In [0]:
output = inferencing_model.predict(X_test)

In [37]:
type(output)

numpy.ndarray

In [38]:
output

array([[2.90404540e-03, 1.43104023e-06, 1.47655461e-04, 2.26350003e-05,
        1.03372156e-04, 4.68166472e-05],
       [7.01217413e-01, 3.01158894e-03, 1.23559520e-01, 1.78430378e-02,
        1.37582034e-01, 2.54184958e-02],
       [5.64030604e-04, 2.36152886e-07, 6.81846723e-05, 4.81286088e-06,
        2.59895569e-05, 1.38065143e-05],
       ...,
       [1.16659775e-01, 1.01646896e-04, 6.93415198e-03, 1.23425294e-03,
        8.39835964e-03, 2.15509627e-03],
       [9.96936679e-01, 3.37710530e-01, 9.55392957e-01, 5.00955917e-02,
        8.54037285e-01, 1.77240700e-01],
       [3.14179953e-04, 8.41999466e-08, 2.80645527e-05, 1.82090162e-06,
        1.04344481e-05, 5.18023307e-06]], dtype=float32)

In [39]:
df = pd.DataFrame(data = output,columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
df

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.002904,1.431040e-06,0.000148,0.000023,0.000103,0.000047
1,0.701217,3.011589e-03,0.123560,0.017843,0.137582,0.025418
2,0.000564,2.361529e-07,0.000068,0.000005,0.000026,0.000014
3,0.000202,5.887585e-08,0.000029,0.000001,0.000008,0.000004
4,0.000239,7.102027e-08,0.000026,0.000001,0.000008,0.000004
...,...,...,...,...,...,...
63973,0.000566,2.185837e-07,0.000049,0.000005,0.000021,0.000011
63974,0.567598,1.080877e-03,0.125369,0.008836,0.099624,0.014385
63975,0.116660,1.016469e-04,0.006934,0.001234,0.008398,0.002155
63976,0.996937,3.377105e-01,0.955393,0.050096,0.854037,0.177241


In [0]:
df.insert(loc=0, column='id', value=test['id'])

In [0]:
df.to_csv('submission.csv')

#### Without TPU

In [0]:
model = mymodel()

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 50)           1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 120)          53280     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 120)          86880     
_________________________________________________________________
global_max_pooling1d (Global (None, 120)               0         
_________________________________________________________________
dropout (Dropout)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                6050  

In [23]:
batch_size = 1576
epochs = 1
hist = model.fit(X_train,train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
143613/143613 [==============================] - 239s 2ms/sample - loss: 0.2160 - accuracy: 0.9398 - val_loss: 0.1429 - val_accuracy: 0.9627


In [24]:
model.predict(X_test)

array([[0.09140462, 0.01085911, 0.04745354, 0.00443642, 0.0454017 ,
        0.00767749],
       [0.09136528, 0.01084439, 0.04746079, 0.00443781, 0.0453743 ,
        0.00768137],
       [0.09135242, 0.01084729, 0.04747661, 0.00444211, 0.04542582,
        0.00768338],
       ...,
       [0.09139293, 0.01084858, 0.04746617, 0.00444338, 0.04538468,
        0.00768317],
       [0.09138866, 0.01084536, 0.04743958, 0.00444103, 0.04541303,
        0.00768201],
       [0.09143018, 0.01085893, 0.04748628, 0.00443559, 0.04538461,
        0.00768657]], dtype=float32)

In [40]:
test

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
2,11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
4,14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
63973,153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
63974,153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
63975,153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
63976,153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0
